In [12]:
import os
import numpy as np
import torch
import torchsummary
from torch import nn
from models import CNN_2d
from train_2d import train_datasets

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cnn_2d = CNN_2d()
cnn_2d = cnn_2d.to(device)
torchsummary.summary(cnn_2d, (1, 512, 512))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 512, 512]             160
         AvgPool2d-2         [-1, 16, 256, 256]               0
            Conv2d-3         [-1, 32, 256, 256]           4,640
         AvgPool2d-4         [-1, 32, 128, 128]               0
            Conv2d-5         [-1, 64, 128, 128]          18,496
         AvgPool2d-6           [-1, 64, 64, 64]               0
            Conv2d-7          [-1, 128, 64, 64]          73,856
         AvgPool2d-8          [-1, 128, 32, 32]               0
            Conv2d-9          [-1, 128, 32, 32]         147,584
        AvgPool2d-10          [-1, 128, 16, 16]               0
           Linear-11                 [-1, 2048]      67,110,912
           Linear-12                 [-1, 2048]       4,196,352
           Linear-13                    [-1, 9]          18,441
Total params: 71,570,441
Trainable para

In [14]:
# Params
batch_size = 20
epochs = 10
filter_method = 'abs_nofit'
batch_size = 20
datasets_root = './datasets'
train_dataset = 'nodemcu-random-train2'
test_dataset = 'nodemcu-random-test2'

def split_batch(batch):
    inputs = np.stack(batch[:, 0], axis=0)[:, :, None]
    targets = np.stack(batch[:, 1], axis=0)
    labels = np.stack(batch[:, 2], axis=0)
    inputs = np.squeeze(inputs, axis=2)
    return inputs, targets, labels

In [15]:
# train 
from batch import get_batch_2d

dataset_names = [train_dataset]

model = cnn_2d

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

batch_c = []  # Batch container
for dataset_name in dataset_names:
    dataset_path = os.path.join(datasets_root, dataset_name)
    dataset_files = list(os.listdir(dataset_path))

    for epoch in range(epochs):
        for i, dataset_file in enumerate(dataset_files):
            if '_traces.npy' in dataset_file:
                for batch in get_batch_2d(dataset_path, dataset_file, batch_c, batch_size=20):

                    input, target, label = split_batch(batch)
                    if np.isnan(input).any():
                        continue


                    xs = torch.tensor(input).float().to(device)
                    ys = torch.tensor(label).float().to(device)

                    y_pred = model(xs)

                    loss = criterion(y_pred, ys)

                    optimizer.zero_grad()
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
                    optimizer.step()
        # saving model every epoch
        torch.save(model.state_dict(), f"models/cnn2d_{epoch}.pt")

print('Training done')

FileNotFoundError: [Errno 2] No such file or directory: './datasets/nodemcu-random-train2'

In [ ]:
# Validate
# load model
from sklearn.metrics import classification_report, confusion_matrix

test_dataset_path = os.path.join(datasets_root, test_dataset)
test_dataset_files = list(os.listdir(test_dataset_path))


for i in range(epoch):
    model.load_state_dict(torch.load(f"models/cnn2d_{i}.pt"))
    true_labels = []
    pred_labels = []
    for i, dataset_file in enumerate(test_dataset_files):
        if '_traces.npy' in dataset_file:
            for batch in get_batch_2d(test_dataset_path, dataset_file, batch_c, batch_size=20):

                input, target, label = split_batch(batch)
                if np.isnan(input).any():
                    continue

                xs = torch.tensor(input).float().to(device)
                ys = torch.tensor(label).float().to(device)

                y_pred = model(xs)

                loss = criterion(y_pred, ys)
                
                true_labels.extend(label)
                pred_labels.extend(y_pred.argmax(dim=1).cpu().numpy())

                print(f'Loss: {loss.item()}')
                
    print(classification_report(true_labels, pred_labels))